# Time series cross validation: Autoregression with seasonal indexes and holidays (with regularization)

This notebook develops Autoregression models to forecast daily Ambulance Response numbers.  

Autoregressive models have the following features:

* They contain lagged dependent variables
* As the model is a regression it can contain season dummy variables at multiple levels in order to capture multiple seasonalitys

Note within this Notebook we will ignore correlated errors.  The most likely impact of this is that prediction intervals
are too narrow.  A less likely, but possible outcome is that there may also be additional information that improve forecating in the errors.

The final section of the notebook analyses prediction interval coverage.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from pandas.plotting import lag_plot
import seaborn as sns

#forecast error metrics
from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)


#'sklearn'
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, LinearRegression

#statsmodels api
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

In [2]:
sm.__version__

'0.11.0'

In [3]:
from amb_forecast.feature_engineering import featurize_time_series

# Data Input

The constants `TOP_LEVEL`, `STAGE`, `REGION`,`TRUST` and `METHOD` are used to control data selection and the directory for outputting results.  

> Output file is `f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv'.csv`.  where metric will be smape, rmse, mase, coverage_80 and coverage_95. Note: `REGION`: is also used to select the correct data from the input dataframe.

In [4]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'elastic-net'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [5]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [6]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [7]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [8]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [9]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#split into train and val AFTER creating new years day.

In [10]:
train.shape

(1828, 9)

In [11]:
test.shape

(365, 9)

# autoregressive lags, seasonal indexes and New years day

Generate lags + new binary categorical feature representing new years day.

In [12]:
#exclude interaction as point forecasts are less accurate.
lagged, calendar_dummies, new_year = featurize_time_series(train[REGION], 
                                                    max_lags=7, 
                                                    include_interactions=False)

In [13]:
#rename column and drop quarters and t from seasonal indexes
new_year.columns = ['new_year']
calendar_dummies = calendar_dummies[calendar_dummies.columns[:-4]]

In [14]:
#combined to single dataframe
processed = pd.concat([train[REGION], lagged, calendar_dummies, new_year], 
                      axis=1)
processed.head()

,actual,Trust_lag1,Trust_lag2,Trust_lag3,Trust_lag4,Trust_lag5,Trust_lag6,Trust_lag7,m_2,m_3,...,m_10,m_11,m_12,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,new_year
actual_dt,,,,,,,,,,,,,,,,,,,,,
2013-12-30,2042.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
2013-12-31,1996.0,2042.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,1,0,0,0,0,0,0
2014-01-01,2570.0,1996.0,2042.0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,1,0,0,0,0,1
2014-01-02,2013.0,2570.0,1996.0,2042.0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,1,0,0,0,0
2014-01-03,2056.0,2013.0,2570.0,1996.0,2042.0,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,1,0,0,0


# Train validation split

In [15]:
#train split into train and validation
train, val = ts_train_test_split(processed, split_date=VAL_SPLIT_DATE)

In [16]:
train.shape

(1279, 26)

In [17]:
val.shape

(549, 26)

## Cross Validation

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).


> Note: prediction uses an iterative method where ground truth inputs are gradually replaced with forecast values.

In [18]:
def time_series_cv(train, val, horizons, lags, step=1, alpha=0.2, 
                   regularised=False, reg_weight=0.1):
    '''
    Time series cross validation across multiple horizons for a autoregressive
    model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Note that when the forecast horizon exceeds the lag number of the model
    forecasts will be based of previous forecasts.  I.e. for a lag 7 model the first 7 
    forecasts will include at least one ground truth observation, but from step 8 onwards all 
    forecasts iwll be based on the previous 7 predicted y values.
    
    Parameters:
    --------
    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days

    step -- step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array - vector of forecast errors from the CVs.
    '''
    cv_preds = []
    cv_actuals = []
    cv_intervals = []
    MAX_LAG = lags
    split = 1
    
    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        print(f'{split}, ', end="")
        split += 1
        
        #create and fit on a new training set (train + val)
        train_cv = np.concatenate([train, val.iloc[:i, 0:]], axis=0) 
        X_train, y_train = train_cv[:,1:], train_cv[:,0]       
        
        # Fit and summarize OLS model
        X_train = sm.add_constant(X_train, prepend=False, has_constant='add')
        model = sm.OLS(endog=y_train, exog=X_train)
        
        if not regularised:
            results = model.fit()
        else:    
            results = model.fit_regularized(method='elastic_net',
                                            alpha=reg_weight, refit=True)

        preds = []
        intervals = []
        current_batch = val.iloc[i, 1:].to_numpy()

        #iteratively predict the next data point
        #and update the lags.  Remember that the
        #actual lags are replaced by FORECAST values 
        #as time goes on.
        for j in range(max(horizons)): 
            
            #one timestep ahead of historical points
            batch_exog = current_batch

            batch_exog = sm.add_constant(batch_exog.reshape(-1,1).T, 
                                         prepend=False,
                                         has_constant='add')
                       
            predictions = results.get_prediction(batch_exog)
            
            df = predictions.summary_frame(alpha=alpha)
            
            y_pred = df['mean'].to_numpy()
            preds.append(y_pred)
            
            #prediction intervals
            y_intervals = df[['obs_ci_lower', 'obs_ci_upper']].to_numpy()
            intervals.append(y_intervals)
            
            #remove tail lagged value add prediction to head 
            current_batch[:MAX_LAG] = np.append(y_pred, current_batch[:MAX_LAG-1])
            
            #get next set of seasonal dummies
            current_batch[MAX_LAG:] = val.iloc[i+j+1, MAX_LAG+1:].to_numpy()
        
        cv_h_preds = []
        cv_h_intervals = []
        cv_test = []
                
        for h in horizons:
            #store the h-step mean prediction
            cv_h_preds.append(np.concatenate(preds[:h]))
            
            #store the h-step prediction intervals
            cv_h_intervals.append(np.concatenate(intervals[:h]))
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h,0])                 
                     
        cv_preds.append(cv_h_preds)
        cv_intervals.append(cv_h_intervals)
        cv_actuals.append(cv_test)
        
    
    print('done.\n')
    return cv_preds, cv_intervals, cv_actuals

## Cross validation (**with** regularization)

Time horizons evaluated (in days) are 7, 14, 28, 56, 84, 365.

Note: These are basic regression models and do not have an ARIMA error process.  This means that interpretation of the coefficients is problematic and should be done with caution.

**All models contain the seasonal dummy variables, t and exceptional days.**

Models evaluated:

* lags 1
* lags 1, 2
* lags 1, 2, 3
* lags 1, 2, 3, .... `MAX_LAGS`

Notes: the cross validation function is returning the error measure over the different time horizons.  So the summary statistics refer to the distrubution of the test statistic.  Not the individual forecast errors of each point.

In [19]:
def split_cv_error(cv_preds, cv_test, error_func):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [20]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

In [21]:


MAX_LAG = 7
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
STEP = 7
PREFIX = 'Trust'

y_lags = [PREFIX+'_lag' + str(i) for i in range(1, MAX_LAG+1)]

select_cv = ['actual'] + y_lags + list(calendar_dummies.columns) + \
    list(new_year.columns)


cv_preds, cv_intervals, cv_test  = time_series_cv(train=train[select_cv][MAX_LAG+1:], 
                                                  val=val[select_cv], 
                                                  horizons=horizons,
                                                  lags=len(y_lags),
                                                  step=STEP,
                                                  alpha=0.2, 
                                                  regularised=True)



split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [22]:
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,3.264534,3.629701,3.854670,4.067859,4.238041,4.367489,4.473390,4.571771,4.646257,4.723101,4.816078,4.904241,4.968646
std,1.200287,1.056591,0.888823,0.829506,0.907699,0.935649,0.935111,0.906036,0.897536,0.895387,0.897089,0.864706,0.245960
min,1.531603,1.861050,2.449089,2.527198,2.565245,2.722428,2.717989,2.724723,2.752764,2.949292,3.186569,3.462700,4.488560
25%,2.441225,2.863237,3.322752,3.302709,3.674638,3.796552,4.099345,4.055521,4.135640,4.239725,4.224898,4.201346,4.745840
50%,3.096386,3.497830,3.743643,4.238427,4.426438,4.470558,4.721285,4.868604,4.809179,4.829757,4.800177,5.061050,5.021345
75%,3.799949,4.175783,4.488454,4.562801,4.866238,5.097712,5.174862,5.213002,5.202153,5.299259,5.498609,5.553106,5.199302
max,6.953349,6.040567,5.429808,5.390963,5.938833,5.783728,5.724250,5.761650,6.117779,6.045818,6.344108,6.561931,5.274809


In [23]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-elastic-net_smape.csv


In [24]:
#CV point predictions rmse - no interactions
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,84.672582,95.801026,103.261059,109.281396,113.651879,116.979000,119.595559,122.094250,123.815270,125.692436,127.835242,129.937203,132.904192
std,29.287007,29.174879,26.462292,24.949116,25.438864,25.206511,24.459545,23.404578,23.107934,23.229848,23.382414,22.703598,5.093162
min,46.346995,59.999095,58.831933,63.799066,64.341211,69.978269,76.028616,76.181564,75.337689,79.541636,86.850600,93.913877,120.788786
25%,64.016862,77.229424,88.204354,92.638312,98.664548,103.127133,110.397491,108.788294,110.007951,112.102011,111.928653,111.416966,128.493728
50%,83.950904,88.392662,98.568948,109.908012,117.676628,116.967411,124.596746,126.967198,125.493156,125.405315,127.281057,128.173934,134.452155
75%,100.560635,108.447312,113.001245,125.555263,132.305288,138.282168,140.912787,140.990855,142.101462,141.576774,147.500462,151.237079,137.435720
max,185.824420,179.924873,163.606330,160.348159,152.167748,152.234559,147.892787,150.161993,160.014482,159.985463,165.347550,170.119965,138.524387


In [25]:
#output rmse
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-elastic-net_rmse.csv


In [26]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train['actual'])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.875557,0.974249,1.034381,1.091215,1.136499,1.171237,1.199963,1.226954,1.247489,1.268792,1.294367,1.318510,1.325109
std,0.344332,0.309016,0.263814,0.248322,0.266788,0.272738,0.272362,0.265703,0.264200,0.264409,0.264985,0.256285,0.065677
min,0.391448,0.471597,0.614331,0.636797,0.649322,0.697388,0.694161,0.695716,0.703499,0.756920,0.820900,0.894166,1.191687
25%,0.632881,0.763286,0.876999,0.854999,0.952392,0.988170,1.068129,1.057499,1.081228,1.110839,1.107718,1.098212,1.267105
50%,0.838845,0.938353,1.007739,1.121042,1.194395,1.209091,1.251750,1.330002,1.313653,1.306614,1.328366,1.355144,1.340336
75%,1.034533,1.121813,1.198767,1.282232,1.328619,1.392184,1.422596,1.417552,1.445886,1.451745,1.508858,1.526987,1.385565
max,2.026483,1.793052,1.507248,1.519510,1.573388,1.538543,1.532167,1.555245,1.669562,1.668598,1.714540,1.782709,1.406119


In [27]:
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-elastic-net_mase.csv


In [28]:
#80% PIs
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.682540,0.608466,0.571429,0.542328,0.522751,0.504409,0.488284,0.472222,0.457966,0.447090,0.433862,0.422399,0.447184
std,0.228755,0.189366,0.158486,0.143555,0.152800,0.156274,0.154278,0.147614,0.142837,0.137699,0.131718,0.124393,0.042405
min,0.142857,0.214286,0.285714,0.321429,0.285714,0.261905,0.265306,0.285714,0.253968,0.271429,0.246753,0.226190,0.394521
25%,0.571429,0.500000,0.476190,0.428571,0.428571,0.380952,0.357143,0.375000,0.380952,0.357143,0.331169,0.339286,0.408219
50%,0.714286,0.642857,0.523810,0.500000,0.485714,0.452381,0.448980,0.446429,0.428571,0.428571,0.402597,0.392857,0.438356
75%,0.857143,0.750000,0.690476,0.625000,0.614286,0.619048,0.591837,0.562500,0.539683,0.507143,0.506494,0.523810,0.490411
max,1.000000,0.928571,0.857143,0.821429,0.828571,0.809524,0.755102,0.767857,0.761905,0.714286,0.688312,0.654762,0.523288


In [29]:
#write 80% coverage to file
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-elastic-net_coverage_80.csv


# Repeat for 95% PIs

In [30]:
#95% PIs
MAX_LAG = 7
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
STEP = 7
PREFIX = 'Trust'

y_lags = [PREFIX+'_lag' + str(i) for i in range(1, MAX_LAG+1)]

select_cv = ['actual'] + y_lags + list(calendar_dummies.columns) + \
    list(new_year.columns)


cv_preds, cv_intervals, cv_test  = time_series_cv(train=train[select_cv][MAX_LAG+1:], 
                                                  val=val[select_cv], 
                                                  horizons=horizons,
                                                  lags=len(y_lags),
                                                  step=STEP,
                                                  alpha=0.05,
                                                  regularised=True)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [31]:
#95% PIs
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.894180,0.846561,0.818342,0.792328,0.769312,0.750441,0.736206,0.721561,0.713110,0.703704,0.693122,0.682540,0.690309
std,0.161394,0.171227,0.148859,0.133277,0.136318,0.136195,0.135008,0.130309,0.129173,0.129034,0.130376,0.123718,0.032045
min,0.428571,0.428571,0.523810,0.571429,0.485714,0.500000,0.510204,0.482143,0.428571,0.471429,0.441558,0.404762,0.652055
25%,0.857143,0.750000,0.714286,0.678571,0.657143,0.642857,0.622449,0.642857,0.650794,0.635714,0.590909,0.571429,0.665753
50%,1.000000,0.928571,0.809524,0.785714,0.771429,0.738095,0.734694,0.714286,0.714286,0.728571,0.714286,0.726190,0.679452
75%,1.000000,1.000000,0.952381,0.910714,0.900000,0.869048,0.846939,0.803571,0.785714,0.771429,0.779221,0.761905,0.723288
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.976190,0.959184,0.946429,0.952381,0.942857,0.909091,0.857143,0.747945


In [32]:
#write 95% coverage to file
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-elastic-net_coverage_95.csv


# End